In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/LocalDocuments/Studia/AlgorytmyWInzynieriiDanych/KM3_Repo`


In [9]:
include("../autodiff/graph.jl")
include("../autodiff/forward.jl")
include("../autodiff/backward.jl")
include("../autodiff/operators.jl")
include("../neuralnet/flux_like_api.jl")

using JLD2
using Random
using Statistics
using Printf

The original test data is too large to upload to GitHub, so we are storing it as `.zip`.

In [3]:
if !isfile("../data/imdb_dataset_prepared.jld2")
  using ZipFile
  r = ZipFile.Reader("../data/imdb_dataset_prepared.jld2.zip")
  for f in r.files
    open("../data/" * f.name, "w") do io
      write(io, read(f))
    end
  end
  close(r)
  println("Extracted imdb_dataset_prepared.jld2 from zip file")
else
  println("imdb_dataset_prepared.jld2 already exists")
end

imdb_dataset_prepared.jld2 already exists


In [4]:
using Flux: DataLoader

X_train = load("../data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../data/imdb_dataset_prepared.jld2", "y_test")

y_train = Float32.(y_train)
y_test = Float32.(y_test)

batch_size = 64

dataset = DataLoader((X_train, y_train), batchsize=batch_size, shuffle=true)

input_neurons = size(X_train, 1)
hidden_neurons = 32
output_neurons = 1

epochs = 5

ϵ = Constant(1e-7)
binary_cross_entropy_loss(y, ŷ) = mean(Constant(-1.0) .* (y .* log.(ŷ .+ ϵ) .+ (Constant(1.0) .- y) .* log.(Constant(1.0) .- ŷ .+ ϵ)))

binary_cross_entropy_loss (generic function with 1 method)

In [5]:
model = Chain(
    Dense(input_neurons, hidden_neurons, relu),
    Dense(hidden_neurons, output_neurons, σ)
)

y = Variable(zeros(1, batch_size), name="y")
x = Variable(zeros(input_neurons, batch_size), name="x")

function loss(model, x, y)
    ŷ = model(x)
    E = binary_cross_entropy_loss(y, ŷ)
    E.name = "loss"
    return E, ŷ
end

otp = setup(Adam(), model)

AdamState(Adam(0.001, 0.9, 0.999, 1.0e-8), Dict{Variable, Tuple{Array, Array, Int64}}(Variable([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], nothing, "bias") => ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0), Variable(Float32[-0.0008089907 0.008288155 … -0.0037912808 0.0027653973; -0.013782425 0.012998216 … 0.0036371658 0.004341168; … ; 0.013890246 0.0071280883 … -0.0058750412 0.0016605166; 0.018025497 0.013208701 … 0.0055700867 -0.0027483099], nothing, "weight") => ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 0), Variable([0.0], nothing, "bias") => ([0.0], [0.0], 0), Variable(Float32[-0.20144412 0.2025319 … 0.2963485 -0.1369667], nothing, "weigh

In [6]:
L, ŷ_node = loss(model, x, y)
graph = topological_sort(L)

for epoch in 1:epochs
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    t = @elapsed begin
        for (xb, yb) in dataset
            current_batch_size = size(xb, 2)

            x.output .= xb
            y.output .= yb

            lval = forward!(graph)

            for param in trainable(model)
                param.gradient = nothing
            end
            backward!(graph)

            update!(otp, model)

            ŷ = ŷ_node.output
            predictions = ŷ .> 0.5
            targets = y.output .> 0.5
            total_correct += count(predictions .== targets)
            total_loss += lval[1] * current_batch_size
            total_samples += current_batch_size
        end
    end

    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (loss: %.4f, acc: %.4f)",
        epoch, t, avg_loss, avg_acc))

end

Epoch: 1 (11.60s) 	Train: (loss: 0.6399, acc: 0.7935)
Epoch: 2 (4.04s) 	Train: (loss: 0.4459, acc: 0.9159)
Epoch: 3 (3.68s) 	Train: (loss: 0.2892, acc: 0.9454)
Epoch: 4 (3.62s) 	Train: (loss: 0.1984, acc: 0.9644)
Epoch: 5 (4.48s) 	Train: (loss: 0.1420, acc: 0.9771)
